In [321]:
from pyelasticsearch.client import ElasticSearch
import json
import requests
import pandas as pd
import math
import numpy as np
import boto3
import re
import collections
from collections import defaultdict
from dateutil import parser
from datetime import datetime

In [580]:
config = json.load(open('../config/configsql.json', 'r'))
aws_access_key_id=config['dev']['aws']['key']
aws_secret_access_key=config['dev']['aws']['secret']
region = 'us-east-1'
endpoint = config['dev']['endpoint']
bucketname=config['dev']['bucketname']['bucketname']
s3r = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
s3c = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)
index_name1 = 'candidates_cv'
index_name2 = 'parsed_resume'
es = ElasticSearch(endpoint, port=443)

In [328]:
d_all = es.search(index = index_name2, query={
                                     "query": {"match_all": {}},
                                     "size":  6
                                     })

In [329]:
for i in range(len(d_all['hits']['hits'])):
    user = d_all['hits']['hits'][i]

In [464]:
for r in range(tdf.shape[0]):
        row = tdf.iloc[r]
        id = row['id']

In [502]:
educations['institution']

['ukrainian national university']

In [501]:
user_record = es.search(index=index_name2,query={"query" : {"match":{"_id": id}}})
user = user_record['hits']['hits'][0]
educations = get_educations(user)

In [476]:
row[2]

'Ukrainian National University'

In [496]:
match_any = False
for edu in educations:
    success = True
    for value in [ "institution", "years" ]: 
        print(edu[value])
        print(row[value])
        if not (edu[value] == row[value]):
            success = False
    if success:
        match_any = True

TypeError: string indices must be integers

In [494]:
match_any = False
for edu in educations:
    success = True
    for value in [ "institution" ]: 
        print(edu[value])
        print(row[2].lower())
        if not (edu['institution'] == row[2].lower()):
            success = False
    if success:
        match_any = True

['ukrainian national university']
ukrainian national university


In [461]:
match_any

True

In [457]:
edu['institution'][0]

'ukrainian national university'

In [450]:
row[2].lower()

'ukrainian national university'

In [487]:
def run_test(es, validation_csv_filename):
    tdf = pd.read_csv(validation_csv_filename)
    tdf = tdf.head()
    tdf.sort_values('id', inplace=True)
    successes = []
    for r in range(tdf.shape[0]):
        row = tdf.iloc[r]
        id = row['id']
        user_record = es.search(index=index_name2,query={"query" : {"match":{"_id": id}}})
        user = user_record['hits']['hits'][0]
        educations = get_educations(user)
        match_any = False
        for edu in educations:
            success = True
            for value in [ "institution", "years" ]: 
                if not (edu[value] == row[value]):
                    success = False
            if success:
                match_any = True
        successes.append(match_any)
    return len(successes) / tdf.shape[0]

In [747]:
def get_educations(parsed_resume):
    education = user['_source']['education']
    if len(user['_source']['education']['entities']) > 0:
        text = [i['Text'] for i in user['_source']['education']['entities']['Entities']]
        type = [i['Type'] for i in user['_source']['education']['entities']['Entities']]
        start_pos = [i['BeginOffset'] for i in user['_source']['education']['entities']['Entities']]
        my_dict = defaultdict(list)
        for k, v in zip(type, text):
            my_dict[k].append(v)
            print(my_dict)
        organizations = my_dict['ORGANIZATION']
        organizations_l = [i.lower() for i in organizations]
        schools = [i for i in organizations_l  if "university" in i]
        years = my_dict['DATE']
        years_low = min(parser.parse(i).year for i in years)
        years_high = max(parser.parse(i).year for i in years)
        my_dict2 = defaultdict(list)

        
        edu_info = {"institution": schools, "years_low":years_low,"years_high":years_high}
        return  edu_info

In [748]:
get_educations(user)

defaultdict(<class 'list'>, {'DATE': ['2017']})
defaultdict(<class 'list'>, {'DATE': ['2017', '2015']})
defaultdict(<class 'list'>, {'DATE': ['2017', '2015'], 'ORGANIZATION': ['Ukrainian National University']})
defaultdict(<class 'list'>, {'DATE': ['2017', '2015', '2012'], 'ORGANIZATION': ['Ukrainian National University']})
defaultdict(<class 'list'>, {'2017': [77]})
defaultdict(<class 'list'>, {'2017': [77], '2015': [163]})
defaultdict(<class 'list'>, {'2017': [77], '2015': [163], 'Ukrainian National University': [229]})
defaultdict(<class 'list'>, {'2017': [77], '2015': [163], 'Ukrainian National University': [229], '2012': [259]})


{'institution': ['ukrainian national university'],
 'years_high': 2017,
 'years_low': 2012}

In [737]:
get_educations(user)

{'institution': ['ukrainian national university'],
 'years_high': 2017,
 'years_low': 2012}

In [734]:
def find_degree_type(univ_pos,user_record):
    values = {"bachelor":'BS',"master":'MS','phd':'phD'}
    for k in user_record:
        if 'education' in k:
            degree_d = {}
            for key in values.keys():
                degree_pos = user_record['education'].lower().find(key)
                degree_d[degree_pos] = key
            degree_type = degree_d.get(univ_pos, degree_d[min(degree_d.keys(), key=lambda k: abs(k-univ_pos))])
            abbr_degree = values[degree_type]
    return abbr_degree

In [739]:
find_degree_type(228,user_record)

'BS'

In [ ]:
def get_educations(user_record):
    # TODO: implement this
    return [{"institution": "?", "degree_type": "?", "degree_level": "?", "years_low": 2004, "years_high": 2015}]

In [542]:
#Just an example how the dictionary may look like
myDict = {'age': ['12'], 'address': ['34 Main Street, 212 First Avenue'],
      'firstName': ['Alan', 'Mary-Ann'], 'lastName': ['Stone', 'Lee']}

def search(values, searchFor):
    for k in values:
        for v in values[k]:
            print(v)
            if searchFor in v:
                return k
    return None

#Checking if string 'Mary' exists in dictionary value
print (search(myDict, 'Mary'))

12
34 Main Street, 212 First Avenue
Alan
Mary-Ann
firstName


In [560]:
id = 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_VIC.DASHKOV@GMAIL.COM_viktor-dashkov-resume.pdf.txt'

In [589]:
bucket = s3.Bucket(bucketname)

In [590]:
objects = bucket.objects.filter(Prefix='resumes/career_page1/2018-04text')

In [610]:
key = []
for obj in objects:
    k = obj.key
    key.append(k)

In [611]:
filenames = []
for i in range(len(key)):
    p = key[i].rfind('/')
    filename = key[i][p+1:]
    filenames.append(filename)

In [621]:
for i in range(len(key)):
    s3c.download_file(bucketname, key[i], filenames[i])

In [603]:
f = open(filenames[9],'r')
s = f.read()
f.close()

In [634]:
fname = 'resumes_career_page1_2018-03_VIC.DASHKOV@GMAIL.COM_viktor-dashkov-resume.pdf.txt'
s3key = 'resumes/career_page1/2018-03text/'+fname
s3c.download_file(bucketname, s3key, fname)

In [635]:
f = open(fname,'r')
s = f.read()
f.close()

In [ ]:
def find_degree_type(univ_pos,text):
    values = {"bachelor":'BS',"master":'MS'}
    degree_pos = text.find('Bachelor')
    

In [647]:
s.find('Bachelor')

2623

In [648]:
s[229:258]

'ing professional with solid e'

In [650]:
user1 = es.search(index=index_name1,query={"query" : {"match":{"_id": id}}})

In [661]:
user2 = user1['hits']['hits'][0]['_source']

In [664]:
user2['education'].find('University')

248

In [670]:
user2['education'].find('University')

248